# LangGraph と AgentCore Memory - Human in the Loop（短期メモリ）

## はじめに
このノートブックでは、Amazon Bedrock AgentCore Memory 機能を LangGraph と統合して **Human-in-the-Loop** ワークフローを作成する方法を紹介します。**短期メモリ**の永続化と、人間の介入のためにエージェント実行を中断する機能を組み合わせ、シームレスなハンドオフを伴う高度なカスタマーサポートシナリオを実現します。

## チュートリアルの詳細

| 情報               | 詳細                                                                            |
|:-------------------|:--------------------------------------------------------------------------------|
| チュートリアルタイプ | 短期会話メモリ                                                                  |
| エージェントのユースケース | 人間へのエスカレーション付きカスタマーサポート                                    |
| エージェントフレームワーク | LangGraph                                                                       |
| LLM モデル          | Anthropic Claude Haiku 4.5                                                     |
| チュートリアルコンポーネント | AgentCore 短期メモリ、LangGraph Checkpointer、Human-in-the-Loop                  |
| 難易度              | 初級                                                                            |

学習内容：
- ワークフロー永続化のための AgentCore Memory を使用したメモリチェックポインターの作成
- Human-in-the-Loop ワークフロー用の LangGraph 中断メカニズムの使用
- 人間の介入のために実行を一時停止できるツールの実装
- LangGraph Command を使用した人間の入力後のエージェントワークフローの再開
- シームレスなハンドオフを伴う複雑なカスタマーサポートシナリオの管理

### シナリオのコンテキスト

この例では、複雑な問題を人間のスーパーバイザーにエスカレーションできる「**カスタマーサポートエージェント**」を作成します。人間の専門知識が必要な状況に遭遇すると、エージェントは実行を一時停止し、現在の状態を AgentCore Memory に保存して、人間の介入を待ちます。その後、人間のスーパーバイザーがガイダンスを提供でき、エージェントは強化されたコンテキストで再開します。

## アーキテクチャ
<div style="text-align:left">
    <img src="images/architecture.png" width="65%" />
</div>

## 前提条件

- Python 3.10以上
- 適切な権限を持つ AWS アカウント
- AgentCore Memory の適切な権限を持つ AWS IAM ロール
- Amazon Bedrock モデルへのアクセス

### 統合の仕組み

Human-in-the-Loop ワークフロー用の LangGraph と AgentCore Memory の統合には以下が含まれます：

1. 永続的な状態管理のためのチェックポインターバックエンドとして AgentCore Memory を使用
2. 特定のポイントで実行を一時停止する中断メカニズムの実装
3. 人間のスーパーバイザーが追加のコンテキストでワークフローを再開できるようにする
4. 中断を超えて会話履歴と状態を維持

このアプローチにより、AI エージェントと人間のスーパーバイザーがシームレスに協働するサポートワークフローが実現します。

環境のセットアップから始めましょう！

In [ ]:
# Install necessary libraries
!pip install -qr requirements.txt

In [ ]:
# Import LangGraph and LangChain components
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langgraph.prebuilt import create_react_agent

# Imports that enable human-in-the-loop implementation
from langgraph.types import Command, interrupt

In [ ]:
import os
import logging

from bedrock_agentcore.memory import MemoryClient
# Import the AgentCoreMemorySaver that we will use as a checkpointer
from langgraph_checkpoint_aws import AgentCoreMemorySaver

logging.getLogger("support-agent").setLevel(logging.INFO)
region = os.getenv('AWS_REGION', 'us-west-2')

logger = logging.getLogger("support-agent")

## ステップ 1: メモリの作成
このセクションでは、AgentCore Memory SDK を使用してメモリストアを作成します。このメモリは LangGraph チェックポインターのバックエンドとして機能し、永続的な Human-in-the-Loop ワークフローを可能にします。

In [ ]:
memory_name = "SupportAgent"

client = MemoryClient(region_name=region)
memory = client.create_or_get_memory(name=memory_name)
memory_id = memory["id"]

### AgentCore Memory の設定

AgentCore Memory チェックポインターを設定し、LLM を初期化しましょう：

- `memory_id` はチェックポイントが保存される AgentCore Memory リソースに対応します
- `region` はリソースの AWS リージョンを指定します
- `MODEL_ID` は LangGraph エージェントを駆動する Bedrock モデルを定義します

`memory_id` と追加の boto3 クライアントキーワード引数（この場合は `region`）を使用してチェックポインターをインスタンス化します。

In [ ]:
MODEL_ID = "global.anthropic.claude-haiku-4-5-20251001-v1:0"

# Initialize checkpointer for state persistence
checkpointer = AgentCoreMemorySaver(memory_id, region_name=region)

## ステップ 2: Human-in-the-Loop ツール
サポートエージェントが使用するツールを定義しましょう。LangGraph の `interrupt` タイプを使用することで、エージェントグラフの実行を中断し、人間が介入してクエリに応答して実行を継続する機会を与えることができます。


In [ ]:
@tool
def human_assistance(query: str) -> str:
    """Request assistance from a human."""
    human_response = interrupt({"query": query})
    return human_response["data"]

@tool
def add(a: int, b: int):
    """Add two integers and return the result"""
    return a + b

@tool
def multiply(a: int, b: int):
    """Multiply two integers and return the result"""
    return a * b


tools = [add, multiply, human_assistance]

## ステップ 3: LangGraph エージェントの実装

AgentCore Memory チェックポインターと Human-in-the-Loop 機能を使用して LangGraph の `create_react_agent` ビルダーでサポートエージェントを作成しましょう：

In [ ]:
# Initialize LLM
llm = init_chat_model(MODEL_ID, model_provider="bedrock_converse", region_name=region)

graph = create_react_agent(
    model=llm,
    tools=tools,
    prompt="You are a helpful assistant",
    checkpointer=checkpointer,
)

graph

## ステップ 4: サポートエージェントの実行
AgentCore Memory チェックポインターと Human-in-the-Loop 統合でエージェントを実行できるようになりました。この例では、明示的にユーザーアシスタンスを要求します。実際には、これは様々な条件でトリガーできます。例えば、特定のキーワードが使用された場合に安全フラグが会話を人間にルーティングすることができます。

### 設定のセットアップ
LangGraph では、config は呼び出し時に必要な属性（ユーザー ID やセッション ID など）を含む `RuntimeConfig` です。[詳細はこちら](https://langchain-ai.github.io/langgraphjs/how-tos/configuration/)を参照してください。

AgentCore Memory チェックポインター（`AgentCoreMemorySaver`）では、以下を指定する必要があります：
- `thread_id`: AgentCore の session_id にマップ（一意の会話スレッド）
- `actor_id`: AgentCore の actor_id にマップ（ユーザー、エージェント、またはその他の識別子）

### グラフ呼び出しの入力
引数 `inputs` には最新のユーザーメッセージのみを渡す必要があります。これには他の状態変数も含めることができますが、シンプルな `create_react_agent` では、メッセージのみが必要です。


In [ ]:
user_input = "I would like to work with a customer service human agent."
config = {"configurable": {"thread_id": "1", "actor_id": "demo-notebook"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

### ワークフローの中断

人間アシスタンスツールが呼び出されたときに実行が一時停止したことに注目してください。現在の状態を検査して、ワークフローがどこで停止したかを確認しましょう：

In [ ]:
snapshot = graph.get_state(config)
snapshot.next

### 人間のスーパーバイザーの介入

ここで人間のスーパーバイザーとして行動し、LangGraph `Command` を使用して応答を送信してワークフローを再開するためのアシスタンスを提供しましょう。AgentCore Memory チェックポインターは会話状態全体を保持しており、チャットを再開できます。

In [ ]:
human_response = (
    "I'm sorry to hear that you are frustrated. Looking at the past conversation history, I can see that you've requested a refund. I've gone ahead and credited it to your account."
)

human_command = Command(resume={"messages": human_response})

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

## まとめ

このノートブックでは、以下を実演しました：

1. Human-in-the-Loop ワークフロー用の AgentCore Memory リソースの作成方法
2. 中断機能を持つ LangGraph エージェントの構築
3. 人間の介入のために実行を一時停止できるツールの実装
4. 中断中のワークフロー状態を永続化するための AgentCoreMemorySaver の使用
5. 人間が提供したコンテキストでのエージェント実行の再開

この統合は、LangGraph の Human-in-the-Loop 機能と AgentCore Memory の堅牢な状態永続化を組み合わせることで、AI エージェントと人間のスーパーバイザーがシームレスに協働する高度なカスタマーサポートワークフローを作成する力を示しています。

ここで実演したアプローチは、多段階のエスカレーション、専門的な人間の専門知識ルーティング、複雑な承認ワークフローなど、より複雑なシナリオに拡張できます。

## クリーンアップ
このノートブックで使用したリソースをクリーンアップするためにメモリを削除しましょう。


In [ ]:
#client.delete_memory_and_wait(memory_id = memory_id, max_wait = 300, poll_interval =10)